# Simulating DGP 0

In [18]:
import sys
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
import os
os.environ["PYTHONHASHSEED"] = "42"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pathlib import Path

project_root = Path.cwd()
while project_root != project_root.parent and not (project_root / "src").exists():
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.simulation.dgp0 import Tier0Config, simulate_panel
from src.simulation.validation import plot_market_plotly
from src.data.feature_eng import feature_eng_syn
from src.model.autoencoder import PriceAutoencoder


#set seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [2]:
cfg = Tier0Config(T=180, burn_in=24)
df = simulate_panel(cfg, n_markets=50, seed=42)

df.head()

,market_id,t,S,c,p
0,0,0,1,0.091785,0.060889
1,0,1,1,0.074484,0.060124
2,0,2,1,0.055169,0.055855
3,0,3,1,0.057500,0.047379
4,0,4,1,0.069277,0.051012


In [3]:
plot_market_plotly(df, market_id=5)

In [4]:
plot_market_plotly(df, 10)

In [5]:
plot_market_plotly(df, market_id= 20)

In [6]:
df_L18 = pd.read_parquet("../data/processed_syn/synth_dgp0_windows_L18.parquet")

df_L18.head()

,market_id,window_start,window_end,window_length,Price 1,Price 2,Price 3,Price 4,Price 5,Price 6,...,Price 14,Price 15,Price 16,Price 17,Price 18,share_C,share_T,share_K,state_mode,is_pure_80
0,0,0,17,18,0.060889,0.060124,0.055855,0.047379,0.051012,0.050917,...,0.007404,-0.000546,-0.008006,-0.021978,-0.019690,0.111111,0.888889,0.0,1,1.0
1,0,1,18,18,0.060124,0.055855,0.047379,0.051012,0.050917,0.055131,...,-0.000546,-0.008006,-0.021978,-0.019690,-0.077646,0.166667,0.833333,0.0,1,1.0
2,0,2,19,18,0.055855,0.047379,0.051012,0.050917,0.055131,0.036140,...,-0.008006,-0.021978,-0.019690,-0.077646,-0.094838,0.222222,0.777778,0.0,1,0.0
3,0,3,20,18,0.047379,0.051012,0.050917,0.055131,0.036140,0.055256,...,-0.021978,-0.019690,-0.077646,-0.094838,-0.104940,0.277778,0.722222,0.0,1,0.0
4,0,4,21,18,0.051012,0.050917,0.055131,0.036140,0.055256,0.060538,...,-0.019690,-0.077646,-0.094838,-0.104940,-0.100550,0.333333,0.666667,0.0,1,0.0


In [7]:
feature_df = feature_eng_syn(df_L18)
feature_df.head()

,market_id,window_start,window_end,window_length,Price 1,Price 2,Price 3,Price 4,Price 5,Price 6,...,CoV_change,zero_change_fraction,AR_1,AR_2,kurtosis_change,max_abs_ret,pos_vol,neg_vol,level_vol,price_range
0,0,0,17,18,0.060889,0.060124,0.055855,0.047379,0.051012,0.050917,...,2.228472,0.117647,-0.077116,0.087097,1.501094,0.029864,0.006910,0.008233,0.029377,0.082866
1,0,1,18,18,0.060124,0.055855,0.047379,0.051012,0.050917,0.055131,...,2.012092,0.058824,-0.182795,0.217825,4.389317,0.057957,0.006910,0.015766,0.038511,0.138185
2,0,2,19,18,0.055855,0.047379,0.051012,0.050917,0.055131,0.036140,...,1.851337,0.058824,-0.020393,0.191116,3.844113,0.057957,0.006910,0.015452,0.046816,0.155376
3,0,3,20,18,0.047379,0.051012,0.050917,0.055131,0.036140,0.055256,...,1.831822,0.058824,-0.019512,0.129449,3.798594,0.057957,0.006910,0.015393,0.053824,0.165478
4,0,4,21,18,0.051012,0.050917,0.055131,0.036140,0.055256,0.060538,...,1.844898,0.058824,-0.049699,0.060755,3.761500,0.057957,0.006829,0.015393,0.058586,0.165478


In [8]:
FEATURES = ["mean_change","volatility","CoV_change",
            "zero_change_fraction","AR_1","kurtosis_change" ,
              "max_abs_ret", "level_vol", "pos_vol", "neg_vol", "AR_2",
               "price_range" ]

pure = feature_df[feature_df["is_pure_80"] == 1]
pure.groupby("state_mode")[FEATURES].mean()

,mean_change,volatility,CoV_change,zero_change_fraction,AR_1,kurtosis_change,max_abs_ret,level_vol,pos_vol,neg_vol,AR_2,price_range
state_mode,,,,,,,,,,,,
0,0.000002,0.034480,47.656607,0.027976,0.069947,0.516971,0.079250,0.054333,0.021622,0.021200,-0.060682,0.182317
1,-0.000022,0.020404,36.881200,0.047527,0.090492,0.306735,0.046291,0.033997,0.012655,0.012334,-0.037001,0.113895
2,0.000047,0.013017,35.270613,0.077360,0.011404,0.203280,0.029022,0.020353,0.007947,0.007914,-0.026028,0.068861


## Traning AE

In [9]:
FEATURES_5 = [
    "volatility", "zero_change_fraction","max_abs_ret",
    "AR_1","price_range"]

X = feature_df[FEATURES_5].to_numpy().astype(np.float32)

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X).astype(np.float32)

#train test split
X_train, X_val = train_test_split(X_scaled, test_size=0.2, random_state=42)

In [10]:
ae = PriceAutoencoder(input_dim=5, latent_dim=2, hidden_dims=(16,8), latent_activation=None)
ae.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")

history = ae.fit(
    X_train, X_train,
    validation_data=(X_val, X_val),
    epochs=200,
    batch_size=256,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

Epoch 1/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8307 - val_loss: 0.5649
Epoch 2/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2737 - val_loss: 0.1752
Epoch 3/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1536 - val_loss: 0.1378
Epoch 4/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1300 - val_loss: 0.1226
Epoch 5/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1191 - val_loss: 0.1147
Epoch 6/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1131 - val_loss: 0.1100
Epoch 7/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1095 - val_loss: 0.1072
Epoch 8/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1074 - val_loss: 0.1055
Epoch 9/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1058 - val_loss: 0.1041
Epoch 10/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1044 - val_loss: 0.1027
Epoch 11/200
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1031 - val_loss: 0.1012
Epoch 12/200
102/102 ━━━━━━━━━━━━━━━━━━━━

## checking embeddings

In [11]:
Z = ae.encoder(X_scaled).numpy()

validation_df = feature_df.copy()

validation_df["z1"] = Z[:, 0]
validation_df["z2"] = Z[:, 1]

In [12]:
#plotting embedding
# Focus on pure windows only for clarity
pure = validation_df[validation_df["is_pure_80"] == 1].copy()

# Map state labels for readability
state_map = {0: "Competitive", 1: "Tacit", 2: "Cartel"}
pure["state_label"] = pure["state_mode"].map(state_map)

fig = px.scatter(
    pure,
    x="z1",
    y="z2",
    color="state_label",
    color_discrete_map={
        "Competitive": "green",
        "Tacit": "orange",
        "Cartel": "red",
    },
    title="Latent Space (Pure Windows Only)",
    opacity=0.6
)

fig.update_layout(template="plotly_white")
fig.show()

### Calculating centriods for states

In [20]:
Z_pure = pure[["z1","z2"]].to_numpy()


#cacluating centroids
mu_C = pure[pure["state_mode"] == 0][["z1","z2"]].mean().to_numpy()
mu_K = pure[pure["state_mode"] == 2][["z1","z2"]].mean().to_numpy()
mu_T = pure[pure["state_mode"] == 1][["z1","z2"]].mean().to_numpy()

#calculating competition direction and scalling 
v = mu_K - mu_C
v = v / np.linalg.norm(v)

In [33]:
#projecting each coordinate on competitoin axis
Z_all = validation_df[["z1","z2"]].to_numpy()
validation_df["conduct_score"] = Z_all @ v
validation_df["conduct_score_centred"] = (Z_all - mu_C) @ v

In [15]:
validation_df.groupby("state_mode")["conduct_score"].mean()

state_mode
0   -0.931807
1   -0.007258
2    0.920156
Name: conduct_score, dtype: float32

In [16]:
validation_df.groupby("state_mode")["conduct_score"].describe()

,count,mean,std,min,25%,50%,75%,max
state_mode,,,,,,,,
0,9572.0,-0.931807,1.248415,-6.545117,-1.656703,-0.862430,-0.328214,7.314342
1,9494.0,-0.007258,1.351561,-5.333056,-0.586736,-0.264665,0.311172,7.592547
2,13534.0,0.920156,1.735206,-5.204399,-0.182823,0.278527,1.806682,12.697027


### Understanding classification

In [26]:
state_map = {0: "Competitive", 1: "Tacit", 2: "Cartel"}

pure["state_label"] = pure["state_mode"].map(state_map)
validation_df["state_label"] = validation_df["state_mode"].map(state_map)

In [37]:
fig = px.scatter(
    pure,
    x="z1",
    y="z2",
    color="state_label",
    opacity=0,
    title="Latent Space (Pure Windows) with Centroids and Conduct Axis",
    template="plotly_white"
)

# Add centroid markers
centroids = np.vstack([mu_C, mu_T, mu_K])
centroid_labels = ["Competitive centroid", "Tacit centroid", "Cartel centroid"]

fig.add_trace(go.Scatter(
    x=centroids[:,0],
    y=centroids[:,1],
    mode="markers+text",
    text=centroid_labels,
    textposition="top center",
    marker=dict(size=14, symbol="x"),
    name="Centroids"
))

# Add arrow from mu_C to mu_K
fig.add_trace(go.Scatter(
    x=[mu_C[0], mu_K[0]],
    y=[mu_C[1], mu_K[1]],
    mode="lines",
    line=dict(width=4, dash="dash"),
    name="Conduct axis (C → K)"
))

# Optional: annotate arrow direction
fig.add_annotation(
    x=mu_K[0], y=mu_K[1],
    ax=mu_C[0], ay=mu_C[1],
    xref="x", yref="y", axref="x", ayref="y",
    showarrow=True, arrowhead=3, arrowsize=1.2, arrowwidth=2,
    text="C→K"
)

fig.show()

### Distribution of conduct score



In [28]:
fig = px.histogram(
    validation_df,
    x="conduct_score_centred",
    color="state_label",
    nbins=60,
    opacity=0.5,
    barmode="overlay",
    title="Centered Conduct Score Distribution by Regime (All Windows)",
    template="plotly_white"
)
fig.show()